In [1]:
import sqlite3
import time
conn = sqlite3.connect('Db-IMDB.db')
c = conn.cursor()

## Work

<img src="db_schema.jpeg">

1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year. 
 
2. List the names of all the actors who played in the movie 'Anand' (1971) 
 
3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.) 
 
4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed. 
 
5.  
a. For each year, count the number of movies in that year that had only female actors. b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer. 
 
6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once. 
 
7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D. 
 
8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies). 
 
9. Find all the actors that made more movies with Yash Chopra than any other director. 
 
10. The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2

In [2]:
import pandas as pd
def execute_query(res):
    data = []
    start = time.time()
    data = res.fetchall()
    print("time taken by query: ", time.time() - start)
    start = time.time()
    col = [d[0] for d in res.description] #Column header names
    df = pd.DataFrame(data, columns=col)
    print("time taken in DF Creation: ", time.time() - start)
    return df

In [3]:
res = c.execute("SELECT * from Movie") #Not putting it in the function intentionally
execute_query(res).head(10)

time taken by query:  0.01599884033203125
time taken in DF Creation:  0.007001399993896484


,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529
4,4,tt8239946,Tumbbad,2018,8.5,7483
5,5,tt7027278,Kedarnath,2018,5.5,1970
6,6,tt3498820,Captain America: Civil War,2016,7.8,536641
7,7,tt8108198,Andhadhun,2018,9.0,18160
8,8,tt3741834,Lion,2016,8.1,170216
9,9,tt6747420,Rajma Chawal,2018,5.7,681


In [4]:
res = c.execute('''select distinct cast(cast((SUBSTR(mout.year, -4, 4)) as number) as integer) year1,(cast((SUBSTR(mout.year, -4, 4)) as number))%3
    from movie mout,m_cast cout where trim(cout.pid)="nm0004435"
    and trim(mout.mid)=trim(cout.mid) 
    order by year1''') #Not putting it in the function intentionally
execute_query(res).head(10)

time taken by query:  0.0
time taken in DF Creation:  0.001035928726196289


,year1,"(cast((SUBSTR(mout.year, -4, 4)) as number))%3"
0,1966,1
1,1967,2
2,1969,1
3,1970,2
4,1971,0
5,1972,1
6,1973,2
7,1974,0
8,1975,1
9,1976,2


## 1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [9]:
# query which will give you the movie and its director
qu = '''SELECT Movie.title, Person.Name, Movie.year FROM Movie, M_Director, Person 
        WHERE Movie.MID=M_Director.MID and Person.PID=M_Director.PID and 
            Movie.MID IN(
                        SELECT MID from M_Genre where GID IN(
                                                SELECT GID FROM Genre WHERE Name LIKE '%Comedy%' 
                                                AND
                                                ((year % 400)=0 OR ((year % 4)=0 AND (year % 100)<>0))>0)
                                                                
                        
                        )'''

# Corrected one below:
qu = '''SELECT Movie.title, Person.Name, Movie.year FROM Movie, M_Director, Person 
        WHERE Movie.MID=M_Director.MID and Person.PID=M_Director.PID 
        AND
        ((year % 400)=0 OR ((year % 4)=0 AND (year % 100)<>0))>0
        AND
        Movie.MID IN(SELECT MID from M_Genre where GID IN(
                                                SELECT GID FROM Genre WHERE Name LIKE '%Comedy%'))
                        
                                                '''

res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  0.006011009216308594
time taken in DF Creation:  0.001996278762817383
(415, 3)


,title,Name,year
0,Mastizaade,Milap Zaveri,2016
1,Mastizaade,Milap Zaveri,2016
2,Harold & Kumar Go to White Castle,Danny Leiner,2004
3,Harold & Kumar Go to White Castle,Danny Leiner,2004
4,Gangs of Wasseypur,Anurag Kashyap,2012


## 2. List the names of all the actors who played in the movie 'Anand' (1971)

In [5]:
qu = '''SELECT Name 
        FROM Person 
        WHERE PID IN (SELECT TRIM(PID) 
                         FROM M_Cast 
                         WHERE MID
                         IN (SELECT MID 
                             FROM Movie 
                             WHERE title = "Anand"));'''

res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  0.08300423622131348
time taken in DF Creation:  0.004010915756225586
(17, 1)


,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo


## 3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [13]:
qu = '''
SELECT Name FROM Person, (SELECT TRIM(PID) castpid From M_Cast WHERE 
    TRIM(MID) IN (SELECT TRIM(MID) FROM Movie WHERE CAST(year AS UNSIGNED)<1970 )
                            ) as temp
 WHERE TRIM(Person.PID) = TRIM(temp.castpid)
 INTERSECT
 SELECT Name FROM Person, (SELECT TRIM(PID) castpid From M_Cast WHERE 
    TRIM(MID) IN (SELECT TRIM(MID) FROM Movie WHERE CAST(year AS UNSIGNED)>1990 )
                            ) as temp
 WHERE TRIM(Person.PID) = TRIM(temp.castpid)
 
'''

# '''
# SELECT Name FROM Person, (
#                             SELECT TRIM(PID) castpid From M_Cast WHERE 
#     TRIM(MID) IN (SELECT TRIM(MID) FROM Movie WHERE CAST(year AS UNSIGNED)<1970 )
#     AND
#     TRIM(MID) IN (SELECT TRIM(MID) FROM Movie WHERE CAST(year AS UNSIGNED)>1990 )
#                             ) as temp
#  WHERE TRIM(Person.PID) = TRIM(temp.castpid)
# '''


res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  0.006999492645263672
time taken in DF Creation:  0.0020020008087158203
(1810, 1)


,Name
0,A.K. Hangal
1,Aachi Manorama
2,Aakash Dabhade
3,Aakash Dahiya
4,Aalaap Majgavkar


## 4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [7]:
qu = ''' SELECT Person.Name, MC_Table.No_of_Movies from Person , (SELECT TRIM(PID) AS NPID, COUNT(MID) AS 'No_of_Movies' 
                                                                  FROM M_Director 
                                                                  GROUP BY PID) as MC_Table
         WHERE Person.PID=MC_Table.NPID 
         ORDER BY MC_Table.No_of_Movies DESC'''

res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  0.006998538970947266
time taken in DF Creation:  0.004999637603759766
(2183, 2)


,Name,No_of_Movies
0,David Dhawan,39
1,David Dhawan,39
2,Mahesh Bhatt,35
3,Mahesh Bhatt,35
4,Ram Gopal Varma,30


<h2>5.
    a. For each year, count the number of movies in that year that had only female actors.<br>
     b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. <br>
     For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.</h2>

### A: For each year, count the number of movies in that year that had only female actors.

In [8]:
# CONSIDERING THE CASE OF 'ONLY FEMALES'

qu = '''
                    SELECT year, 
                    TRIM(MID) AS Female_Movie_MID 
                    FROM Movie 
                    WHERE MID IN(
                                SELECT Movie.MID
                                FROM Movie
                                WHERE MID
                                NOT IN(
                                        SELECT MID
                                        FROM M_Cast, Person
                                        WHERE TRIM(Person.PID)=TRIM(M_Cast.PID)
                                              AND
                                              Person.Gender='Male'
                                        )
                                )
                    GROUP BY year
                       
        
'''



res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  0.0
time taken in DF Creation:  0.0020041465759277344
(7, 2)


,year,Female_Movie_MID
0,1939,tt0375882
1,1999,tt0272001
2,2000,tt0354922
3,2009,tt1587388
4,2012,tt2355921


##### B. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. 

In [9]:
# WE SHOULD TRY THIS TOO...
qu = '''SELECT m.year,
        COUNT(Female_Movie_MID) AS Female_Only_Movies_Count,
        COUNT(TRIM(MID)) AS Total_Movies_Count,
        CAST(COUNT(Female_Movie_MID)/COUNT(TRIM(MID)) AS DECIMAL(10, 4)) AS Percentage_Movie_Count
        FROM Movie m
        LEFT JOIN
        (
                    SELECT year, 
                    TRIM(MID) AS Female_Movie_MID 
                    FROM Movie 
                    WHERE MID IN(
                                SELECT Movie.MID
                                FROM Movie
                                WHERE MID
                                NOT IN(
                                        SELECT MID
                                        FROM M_Cast, Person
                                        WHERE TRIM(Person.PID)=TRIM(M_Cast.PID)
                                              AND
                                              Person.Gender='Male'
                                        )
                                )
        ) c
        ON m.MID = c.Female_Movie_MID
        GROUP BY m.year
        
'''



res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  0.0029981136322021484
time taken in DF Creation:  0.0040051937103271484
(125, 4)


,year,Female_Only_Movies_Count,Total_Movies_Count,Percentage_Movie_Count
0,1931,0,1,0
1,1936,0,3,0
2,1939,1,2,0
3,1941,0,1,0
4,1943,0,1,0


## 6. Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once. 

In [10]:
# query which will give you the movie and its director
qu = '''
    SELECT Movie.title,MMID, Max_Cast_Size FROM Movie, (SELECT MID AS MMID, MAX(Cast_Size) AS Max_Cast_Size FROM(SELECT MID, COUNT(TRIM(PID)) AS Cast_Size FROM M_Cast GROUP BY TRIM(MID))) where TRIM(Movie.MID) = TRIM(MMID) 
'''
res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  0.004004240036010742
time taken in DF Creation:  0.003002643585205078
(1, 3)


,title,MMID,Max_Cast_Size
0,Ocean's Eight,tt5164214,238


## 7. A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D. 

In [11]:
# query which will give you the movie and its director
# qu = '''
# SELECT RowNum,AbsRowNum, MAX(Movies_released), MIN(only_year)||"-"|| MAX(only_year) AS Decade
# FROM (SELECT SUBSTR(year, -4, 4) only_year, 
#       COUNT(MID) AS Movies_released 
#       FROM Movie 
#       GROUP BY only_year 
#       ORDER BY only_year ASC) Movie_agg 
#       LEFT JOIN (SELECT ROW_NUMBER() 
#                  OVER(ORDER BY Distinct_MID)+(SELECT MIN(year) FROM Movie)-1 AS RowNum,
#                  (ROW_NUMBER() OVER(ORDER BY Distinct_MID))/10 AS AbsRowNum
#                  FROM (SELECT 
#                        DISTINCT(MID) AS Distinct_MID 
#                        FROM Movie 
#                        LIMIT (2018) - (1930))) AS COUNT_YEARS 
#                        ON TRIM(Movie_agg.only_year) = TRIM(COUNT_YEARS.RowNum)
#        GROUP BY AbsRowNum                
#                       '''

qu = '''
SELECT MAX(Decade) AS MAX_DECADE, MAX_MOVIE_RELEASED 
FROM(
SELECT RowNum,AbsRowNum, MAX(Movies_released) MAX_MOVIE_RELEASED, MIN(only_year)||"-"|| MAX(only_year) AS Decade
FROM (SELECT SUBSTR(year, -4, 4) only_year, 
      COUNT(MID) AS Movies_released 
      FROM Movie 
      GROUP BY only_year 
      ORDER BY only_year ASC) Movie_agg 
      LEFT JOIN (SELECT ROW_NUMBER() 
                 OVER(ORDER BY Distinct_MID)+(SELECT MIN(year) FROM Movie)-1 AS RowNum,
                 (ROW_NUMBER() OVER(ORDER BY Distinct_MID))/10 AS AbsRowNum
                 FROM (SELECT 
                       DISTINCT(MID) AS Distinct_MID 
                       FROM Movie 
                       LIMIT (2018) - (1930))) AS COUNT_YEARS 
                       ON TRIM(Movie_agg.only_year) = TRIM(COUNT_YEARS.RowNum)
       GROUP BY AbsRowNum
       )
                      '''


res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  0.0
time taken in DF Creation:  0.0020020008087158203
(1, 2)


,MAX_DECADE,MAX_MOVIE_RELEASED
0,2010-2018,136


## 8. Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies). 

In [12]:

qu = '''
SELECT DISTINCT(TRIM(PID)) AS MPID, name FROM Person P
    WHERE exists
        (WITH TEMP(
 PID, year, MID
 )AS(
 
        SELECT PID, CAST(CAST (SUBSTR(year, -4, 4) AS NUMERIC(19,4)) AS INT) AS year, COUNT(TRIM(Movie.MID)) AS MID 
        FROM M_Cast, Movie 
        WHERE TRIM(M_Cast.MID) = Movie.MID
              AND
              TRIM(M_Cast.PID) = TRIM(P.PID)
        GROUP BY SUBSTR(TRIM(year), -4, 4)
                                            
 )
 SELECT  
            PID,  year,
            MID, 
            NextYear, 
            (NextYear - SUBSTR(TRIM(year), -4, 4)) AS Diff
            FROM 
            (SELECT PID, year, MID, (
                                        SELECT MIN(year)
                                        FROM TEMP T2
                                        WHERE T2.PID = T1.PID
                                        AND 
                                        T2.year > T1.year
                                    ) AS NextYear
            FROM TEMP AS T1
                
            ) where Diff > 3)
        

 

'''

res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head()

time taken by query:  6298.947175741196
time taken in DF Creation:  0.003000497817993164
(4382, 2)


,MPID,Name
0,nm2951768,Freida Pinto
1,nm4575116,Rohan Chand
2,nm0001162,Griffin Dunne
3,nm0949433,Damian Young
4,nm1753302,Waris Ahluwalia


## 9. Find all the actors that made more movies with Yash Chopra than any other director. 

In [13]:
# #QUERY TO FETCH PERSON ID OF YASH CHOPRA
# qu = '''
#     (SELECT TRIM(PID) FROM Person WHERE Name='Yash Chopra')
# '''
# # QUERY TO FETCH MOVIE (MID) DIRECTED BY YC FROM HIS PID
# qu = '''
#     SELECT MID FROM M_Director WHERE TRIM(PID) =  (SELECT TRIM(PID) FROM Person WHERE Name='Yash Chopra')
# '''
# # QUERY TO FETCH CASTS(PID) WHO WORKED IN THE MOVIES DIRECTED BY YC
# qu = '''
#     SELECT PID FROM M_Cast WHERE TRIM(MID) IN(SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) =  (SELECT TRIM(PID) FROM Person WHERE Name='Yash Chopra')) 
# '''

# # FROM SAME ABOVE QUERY DO GROUP BY TO COUNT NUMBER OF MOVIES THEY HAVE DONE WITH YC.
# qu = '''
#     SELECT PID, COUNT(MID) AS Movies_with_YC FROM M_Cast WHERE TRIM(MID) 
#     IN(SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) =  (SELECT TRIM(PID) FROM Person WHERE Name='Yash Chopra')) 
#     GROUP by PID
# '''

# # PRINTING THOSE CASTS WHO MADE MORE THAN 5 MOVIES WITH YC
# qu = '''
#     SELECT PID, Movies_with_YC FROM(
#     SELECT PID, COUNT(MID) AS Movies_with_YC FROM M_Cast WHERE TRIM(MID) 
#     IN(SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) =  (SELECT TRIM(PID) FROM Person WHERE Name='Yash Chopra')) 
#     GROUP by PID 
#     ) WHERE Movies_with_YC > 5
    
# ''' # KEEPING PID - nm0000821 FOR FUTURE PURPOSES

# # REDUNDANT DELETE IT
# # FROM SAME ABOVE QUERY DO GROUP BY TO COUNT NUMBER OF MOVIES THEY HAVE DONE WITH YC.
# qu = '''
#     SELECT PID, MID,  COUNT(MID) AS Movies_with_YC FROM M_Cast WHERE TRIM(MID) 
#     IN(SELECT TRIM(MID) FROM M_Director WHERE TRIM(PID) =  (SELECT TRIM(PID) AS YCPID FROM Person WHERE Name='Yash Chopra')) 
#     GROUP by PID
# '''
# # FROM SAME ABOVE QUERY PRINTING DIRECTOR NAME AND COUNT
# # we have a table having casts worked with only yc and their movie counts.
# qu = '''
#     SELECT YC_Movies.PID AS YC_CastID,
#     M_Director.PID AS DirectorID,
#     YC_Movies.Movies_with_YC
#     FROM M_Director,(SELECT PID, MID,  COUNT(MID) AS Movies_with_YC 
#                      FROM M_Cast 
#                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                         FROM M_Director 
#                                         WHERE TRIM(PID) =  (SELECT 
#                                                             TRIM(PID) AS YCPID 
#                                                             FROM Person 
#                                                             WHERE Name='Yash Chopra')
#                                         )GROUP by PID
#                     ) AS YC_Movies
#     WHERE M_Director.MID = YC_Movies.MID
# '''
# #(FOR NEXT QUERY REMOVING GROUP BY FROM 52 WE ARE NOT USING COUNT)

# # now we have to create one more query which will target the innermost subquery
# # SELECT TRIM(PID) AS YCPID FROM Person WHERE Name='Yash Chopra'

# # query to fetch other directors
# # qu = '''
# # SELECT DISTINCT(Person.PID) AS Other_Directors_ID FROM M_Director, Person WHERE Person.Name!="Yash Chopra AND Person.PID = M_Director.PID
# # '''

# # table to fetch the casts working with different directors and there movie count.
# # using two group by one on movies and other on directors
# qu = '''
#     SELECT YC_Movies.PID AS YC_CastID,
#     M_Director.PID AS DirectorID,
#     COUNT(YC_Movies.MovieIdWithYC) AS MovieCount
#     FROM M_Director, (SELECT PID, MID AS MovieIdWithYC
#                      FROM M_Cast 
#                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                         FROM M_Director 
#                                         WHERE TRIM(PID) IN  (SELECT 
#                                                             TRIM(DISTINCT(Person.PID)) AS Other_Directors_ID 
#                                                             FROM M_Director, Person 
#                                                             WHERE Person.Name!="Yash Chopra" 
#                                                                         AND 
#                                                             Person.PID = M_Director.PID)
#                                         )
#                     ) AS YC_Movies
#     WHERE M_Director.MID = YC_Movies.MovieIdWithYC
#     GROUP BY DirectorID, YC_CastID
# '''

# #Checking the movie 
# qu = '''
#     SELECT YC_CastID, DirectorID, MovieCount FROM (
#                                                     SELECT NotYC_Movies.PID AS YC_CastID,
#                                                     M_Director.PID AS DirectorID,
#                                                     COUNT(NotYC_Movies.MovieIdWithYC) AS MovieCount
#                                                     FROM M_Director, (SELECT PID, MID AS MovieIdWithYC
#                                                                      FROM M_Cast 
#                                                                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                                                                         FROM M_Director 
#                                                                                         WHERE TRIM(PID) IN  (SELECT 
#                                                                                                             TRIM(DISTINCT(Person.PID)) AS Other_Directors_ID 
#                                                                                                             FROM M_Director, Person 
#                                                                                                             WHERE Person.Name!="Yash Chopra" 
#                                                                                                                         AND 
#                                                                                                             Person.PID = M_Director.PID)
#                                                                                         )
#                                                                     ) AS NotYC_Movies
#                                                     WHERE M_Director.MID = NotYC_Movies.MovieIdWithYC
#                                                     GROUP BY DirectorID, YC_CastID
#     ) WHERE MovieCount > 10
#     ORDER BY MovieCount DESC
    
# '''

# #Checking the movie 
# qu = '''
#     SELECT NYC_CastID, DirectorID, MovieCount_NYC, Movies_with_YC, YCDirectorID FROM (
#                                                     SELECT NotYC_Movies.PID AS NYC_CastID,
#                                                     M_Director.PID AS DirectorID,
#                                                     COUNT(NotYC_Movies.MovieIdWithNYC) AS MovieCount_NYC
#                                                     FROM M_Director, (SELECT PID, MID AS MovieIdWithNYC
#                                                                      FROM M_Cast 
#                                                                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                                                                         FROM M_Director 
#                                                                                         WHERE TRIM(PID) IN  (SELECT 
#                                                                                                             TRIM(DISTINCT(Person.PID)) AS Other_Directors_ID 
#                                                                                                             FROM M_Director, Person 
#                                                                                                             WHERE Person.Name!="Yash Chopra" 
#                                                                                                                         AND 
#                                                                                                             Person.PID = M_Director.PID)
#                                                                                         )
#                                                                     ) AS NotYC_Movies
#                                                     WHERE M_Director.MID = NotYC_Movies.MovieIdWithNYC
#                                                     GROUP BY DirectorID, NYC_CastID
#     )
#     LEFT JOIN (SELECT YC_Movies.PID AS CastID,
#                     M_Director.PID AS YCDirectorID,
#                     YC_Movies.Movies_with_YC
#                     FROM M_Director,(SELECT PID, MID,  COUNT(MID) AS Movies_with_YC 
#                                      FROM M_Cast 
#                                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                                         FROM M_Director 
#                                                         WHERE TRIM(PID) =  (SELECT 
#                                                                             TRIM(PID) AS YCPID 
#                                                                             FROM Person 
#                                                                             WHERE Name='Yash Chopra')
#                                                         )GROUP by PID
#                                     ) AS YC_Movies
#                     WHERE M_Director.MID = YC_Movies.MID
#                     )
#     ON NYC_CastID = CastID
    
# '''

# #NOW WE WILL FILTER THOSE CASTS WHO DID MORE MOVIES WITH YASH CHOPRA THAN ANY OTHER DIRECTOR

# qu = '''
# SELECT NYC_CastID, DirectorID, MovieCount_NYC, Movies_with_YC, YCDirectorID 
# FROM ( SELECT NYC_CastID, DirectorID, MovieCount_NYC, Movies_with_YC, YCDirectorID FROM (
#                                                     SELECT NotYC_Movies.PID AS NYC_CastID,
#                                                     M_Director.PID AS DirectorID,
#                                                     COUNT(NotYC_Movies.MovieIdWithNYC) AS MovieCount_NYC
#                                                     FROM M_Director, (SELECT PID, MID AS MovieIdWithNYC
#                                                                      FROM M_Cast 
#                                                                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                                                                         FROM M_Director 
#                                                                                         WHERE TRIM(PID) IN  (SELECT 
#                                                                                                             TRIM(DISTINCT(Person.PID)) AS Other_Directors_ID 
#                                                                                                             FROM M_Director, Person 
#                                                                                                             WHERE Person.Name!="Yash Chopra" 
#                                                                                                                         AND 
#                                                                                                             Person.PID = M_Director.PID)
#                                                                                         )
#                                                                     ) AS NotYC_Movies
#                                                     WHERE M_Director.MID = NotYC_Movies.MovieIdWithNYC
#                                                     GROUP BY DirectorID, NYC_CastID
#     )
#     LEFT JOIN (SELECT YC_Movies.PID AS CastID,
#                     M_Director.PID AS YCDirectorID,
#                     YC_Movies.Movies_with_YC
#                     FROM M_Director,(SELECT PID, MID,  COUNT(MID) AS Movies_with_YC 
#                                      FROM M_Cast 
#                                      WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                                                         FROM M_Director 
#                                                         WHERE TRIM(PID) =  (SELECT 
#                                                                             TRIM(PID) AS YCPID 
#                                                                             FROM Person 
#                                                                             WHERE Name='Yash Chopra')
#                                                         )GROUP by PID
#                                     ) AS YC_Movies
#                     WHERE M_Director.MID = YC_Movies.MID
#                     )
#     ON NYC_CastID = CastID
#     WHERE 
#     Movies_with_YC > MovieCount_NYC
# )
# '''

# AS WE HAVE USED AGGREGATE FUNCTION THESE ROWS ARE UNIQUE
# JOINING THE QUERY WITH Person Table TO FETCH THE CAST NAME
qu = '''
SELECT DISTINCT(TRIM(NYC_CastID)) AS NYC_CastID, name, MovieCount_NYC, Movies_With_YC FROM(
SELECT NYC_CastID, DirectorID, MovieCount_NYC, MAX(Movies_with_YC) AS Movies_With_YC, YCDirectorID 
FROM ( SELECT NYC_CastID, DirectorID, MovieCount_NYC, Movies_with_YC, YCDirectorID FROM (
                                                    SELECT NotYC_Movies.PID AS NYC_CastID,
                                                    M_Director.PID AS DirectorID,
                                                    COUNT(NotYC_Movies.MovieIdWithNYC) AS MovieCount_NYC
                                                    FROM M_Director, (SELECT TRIM(PID) AS PID, TRIM(MID) AS MovieIdWithNYC
                                                                     FROM M_Cast 
                                                                     WHERE TRIM(MID) IN(SELECT TRIM(MID) 
                                                                                        FROM M_Director 
                                                                                        WHERE TRIM(PID) IN  (SELECT 
                                                                                                            TRIM(DISTINCT(TRIM(Person.PID))) AS Other_Directors_ID 
                                                                                                            FROM M_Director, Person 
                                                                                                            WHERE Person.Name!="Yash Chopra" 
                                                                                                                        AND 
                                                                                                            TRIM(Person.PID) = TRIM(M_Director.PID))
                                                                                        )
                                                                    ) AS NotYC_Movies
                                                    WHERE TRIM(M_Director.MID) = TRIM(NotYC_Movies.MovieIdWithNYC)
                                                    GROUP BY TRIM(DirectorID), TRIM(NYC_CastID)
    )
    LEFT JOIN (SELECT TRIM(YC_Movies.PID) AS CastID,
                    TRIM(M_Director.PID) AS YCDirectorID,
                    YC_Movies.Movies_with_YC
                    FROM M_Director,(SELECT TRIM(PID) AS PID, TRIM(MID) AS MID,  COUNT(TRIM(MID)) AS Movies_with_YC 
                                     FROM M_Cast 
                                     WHERE TRIM(MID) IN(SELECT TRIM(MID) 
                                                        FROM M_Director 
                                                        WHERE TRIM(PID) =  (SELECT 
                                                                            TRIM(PID) AS YCPID 
                                                                            FROM Person 
                                                                            WHERE Name='Yash Chopra')
                                                        )GROUP by PID
                                    ) AS YC_Movies
                    WHERE TRIM(M_Director.MID) = TRIM(YC_Movies.MID)
                    )
    ON TRIM(NYC_CastID) = TRIM(CastID)
    WHERE 
    Movies_with_YC > MovieCount_NYC
) GROUP BY NYC_CastID
ORDER BY Movies_With_YC DESC
), Person
WHERE TRIM(NYC_CastID) = TRIM(Person.PID)




'''

res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  1.2762513160705566
time taken in DF Creation:  0.03500080108642578
(85, 4)


,NYC_CastID,Name,MovieCount_NYC,Movies_With_YC
0,nm0707271,Jagdish Raj,2,11
1,nm0471443,Manmohan Krishna,1,10
2,nm0471443,Manmohan Krishna,1,10
3,nm0407002,Iftekhar,1,9
4,nm0025630,Vikas Anand,1,8


## 10. The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2

In [14]:
# FIRST  WE CHECK DISTINCT ARTIST IN M_CAST TABLE
qu = '''
        SELECT DISTINCT(TRIM(PID)) AS PID FROM M_Cast
    '''


res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  0.3670022487640381
time taken in DF Creation:  0.008018016815185547
(32128, 1)


,PID
0,nm0000288
1,nm0000949
2,nm1212722
3,nm0365140
4,nm0785227


##### There are total 32128 artist present in the db

In [15]:
# query which will give you the movie and its director
# qu = ''' SELECT PID, Name FROM Person WHERE Name LIKE '% Master Shahrukh%'                        
#      '''
# # DEGREE 1
# qu = '''SELECT TRIM(PID) 
#         FROM M_Cast 
#         WHERE TRIM(MID) IN(SELECT TRIM(MID) 
#                      FROM M_Cast 
#                      WHERE TRIM(PID) = (SELECT TRIM(PID) 
#                                         FROM Person 
#                                         WHERE Name 
#                                         LIKE '%Master Shahrukh%')
#         )
#     '''
# DEGREE 2 
qu = '''
        SELECT Name FROM Person, 
        (
        SELECT DISTINCT(TRIM(PID)) PID 
        FROM M_Cast 
        WHERE TRIM(MID) IN(
                    SELECT DISTINCT(TRIM(MID)) 
                     FROM M_Cast 
                     WHERE TRIM(PID) IN(
                     SELECT TRIM(PID) 
                    FROM M_Cast 
                    WHERE TRIM(MID) IN(SELECT DISTINCT(TRIM(MID)) 
                                        FROM M_Cast 
                                        WHERE TRIM(PID) = (SELECT DISTINCT(TRIM(PID)) 
                                                            FROM Person 
                                                            WHERE Name 
                                                            LIKE '%Master Shahrukh%')
                                        ) 
                             )
                    ) 
        ) AS T1
        WHERE TRIM(Person.PID)= TRIM(T1.PID)
    '''



res = c.execute(qu)
df = execute_query(res)
print(df.shape)
df.head(5)

time taken by query:  243.73374199867249
time taken in DF Creation:  0.0030040740966796875
(14675, 1)


,Name
0,Freida Pinto
1,Caroline Christl Long
2,Rajeev Pahuja
3,Michelle Santiago
4,Jandre le Roux
